# Understanding and optimizing customer datasets

In this lesson, we will be focusing on the Google Merchandise Store's (GStore) customer dataset. This dataset contains information about the session history of each user on the GStore's website along with their generated revenue in case they ever made a purchase.

<img src="images/google_store.jpg" width="400">

Our goal with this lesson is to learn what kind of data we should be collecting for our own business in order to properly apply business analytics and intelligence.

Since most of the data fields present in this dataset is based off of Google Analytics along with some internal transactional data, any business can apply the techniques taught in this course to harvest and optimize such datasets.

The dataset contains the following information,

- fullVisitorId - A unique identifier for each user of the Google Merchandise Store.
- channelGrouping - The channel via which the user came to the Store.
- date - The date on which the user visited the Store.
- device - The specifications for the device used to access the Store.
- geoNetwork - This section contains information about the geography of the user.
- socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
- totals - This section contains aggregate values across the session.
- trafficSource - This section contains information about the Traffic Source from which the session originated.
- visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
- visitNumber - The session number for this user. If this is the first session, then this is set to 1.
- visitStartTime - The timestamp (expressed as POSIX time).
- hits - This row and nested fields are populated for any and all types of hits. Provides a record of all page visits.
- customDimensions - This section contains any user-level or session-level custom dimensions that are set for a session. This is a repeated field and has an entry for each dimension that is set.
- totals - This set of columns mostly includes high-level aggregate data.

Let us start by importing the necessary libraries,

In [1]:
import pandas as pd

Next, importing the CSV file called `gstore_data.csv` which contains information about each user's website visit along with the revenue they generated for Google.

In [2]:
# Reading in the CSV file as a DataFrame
store_df = pd.read_csv('data/gstore_data.csv', low_memory=False)

In [3]:
# Looking at the first five rows
store_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,not available in demo dataset,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let us look at the shape of the DataFrame to determine how many sessions were recorded on the GStore's website.

In [4]:
# Viewing the shape of the DataFrame
store_df.shape

(903653, 55)

Since the dataset is large, let us see how much memory it occupies.

In [5]:
# Printing a concise summary of the DataFrame
store_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   channelGrouping                                    903653 non-null  object 
 1   date                                               903653 non-null  int64  
 2   fullVisitorId                                      903653 non-null  object 
 3   sessionId                                          903653 non-null  object 
 4   socialEngagementType                               903653 non-null  object 
 5   visitId                                            903653 non-null  int64  
 6   visitNumber                                        903653 non-null  int64  
 7   visitStartTime                                     903653 non-null  int64  
 8   device.browser                                     903653 non-null  object

**Should we remove the rows (session history) to reduce memory usage?**

We should keep in mind that the session history is very important for profiling the customers of a business, along with their behavioral patterns.

Therefore, even if the dataset feels large, we should not remove the rows in the dataset as long as possible. Alternatively, we can take a sample out of the dataset that is most representative of the entire dataset in order to conduct our analysis. This is what most statisticians do.

But, since we will be performing Machine Learning later on in this course, let us keep all the rows of data for our analysis.

**Should we remove the columns (data fields) to reduce memory usage?**

The removal of a column is completely dependent on the data it holds. If a column doesn't contain data that is significant to analysis or modeling then it can be removed. Examples of such columns are columns containing mostly null values, a single constant value or a randomly generated value.

First, let us print out all the column names in the dataset.

In [6]:
# Printing the column names
store_df.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserVersion', 'device.browserSize',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.isMobile', 'device.mobileDeviceBranding',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.flashVersion', 'device.language', 'device.screenColors',
       'device.screenResolution', 'device.deviceCategory',
       'geoNetwork.continent', 'geoNetwork.subContinent', 'geoNetwork.country',
       'geoNetwork.region', 'geoNetwork.metro', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.networkDomain', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
       'totals.hits', 'totals.pageviews', 'totals.bounces', 'totals.newVisits',
       'totals.t

We can select any of the column and print out the unique values in it by using the `nunique()` method.

In [7]:
# Count the unique elements in the column
store_df['channelGrouping'].nunique(dropna=False)

8

We can also print out the unique values for all the columns at once.

In [8]:
# Count the unique elements in the DataFrame
store_df.nunique(dropna=False)

channelGrouping                                           8
date                                                    366
fullVisitorId                                        714167
sessionId                                            902755
socialEngagementType                                      1
visitId                                              886303
visitNumber                                             384
visitStartTime                                       887159
device.browser                                           54
device.browserVersion                                     1
device.browserSize                                        1
device.operatingSystem                                   20
device.operatingSystemVersion                             1
device.isMobile                                           2
device.mobileDeviceBranding                               1
device.mobileDeviceModel                                  1
device.mobileInputSelector              

There are a lot of columns contaning only a single value. So, let us fetch them out programmatically by appending them into an empty list, 

In [9]:
# Creating an empty list
single_val_cols = []

# Looping across all column names
for x in store_df.columns:
    if store_df[x].nunique(dropna=False) == 1:
        single_val_cols.append(x)

In [10]:
single_val_cols

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.operatingSystemVersion',
 'device.mobileDeviceBranding',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.flashVersion',
 'device.language',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

These columns may contain some kind of data value, however, the information in them is non-existent since they do not help to distinguish one user's session history from the other. So, let us remove them using the `drop()` method off of the DataFrame.

In [11]:
# Dropping the columns
store_df.drop(single_val_cols, axis=1, inplace=True)

In [12]:
# Looking at the first five rows
store_df.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,device.isMobile,...,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,Windows,False,...,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,Macintosh,False,...,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,Windows,False,...,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,Linux,False,...,google + online,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,Android,True,...,(not provided),True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


If we look at the columns again, `sessionId` seems to be randomly generated for each session irrespective of the user. Therefore, we can drop it as well.

In [13]:
# Dropping the column
store_df.drop(['sessionId'], axis=1, inplace=True)

Finally, let us check again if we have left out any column that doesn't add value to us.

In [14]:
# Count the unique elements in the DataFrame
store_df.nunique(dropna=False)

channelGrouping                                      8
date                                               366
fullVisitorId                                   714167
visitId                                         886303
visitNumber                                        384
visitStartTime                                  887159
device.browser                                      54
device.operatingSystem                              20
device.isMobile                                      2
device.deviceCategory                                3
geoNetwork.continent                                 6
geoNetwork.subContinent                             23
geoNetwork.country                                 222
geoNetwork.region                                  376
geoNetwork.metro                                    94
geoNetwork.city                                    649
geoNetwork.networkDomain                         28064
totals.hits                                        274
totals.pag

Looks like all the columns are useful. Now, let us check the memory usage again.

In [15]:
# Printing a concise summary of the DataFrame
store_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 35 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   channelGrouping                               903653 non-null  object 
 1   date                                          903653 non-null  int64  
 2   fullVisitorId                                 903653 non-null  object 
 3   visitId                                       903653 non-null  int64  
 4   visitNumber                                   903653 non-null  int64  
 5   visitStartTime                                903653 non-null  int64  
 6   device.browser                                903653 non-null  object 
 7   device.operatingSystem                        903653 non-null  object 
 8   device.isMobile                               903653 non-null  bool   
 9   device.deviceCategory                         90

Great! That's a huge improvement.

Looking at the data fields we can see that the dataset can be grouped onto various categories such as:

- Session Visit Information (channelGrouping, date, fullVisitorId, ...)

- Device Information (device.browser, device.operatingSystem, device.isMobile, ...)

- Geographical Information (geoNetwork.continent, geoNetwork.subContinent, geoNetwork.country, ...)

- Aggregated Information (totals.hits, totals.pageviews, totals.bounces, ...)

- Traffic Source Information (trafficSource.campaign, trafficSource.source, trafficSource.medium, ...)

These will be our features for predicting the Revenue Information which is given by the column name `totals.transactionRevenue`.

Finally, let us export this optimized DataFrame to a new CSV file so that we can use it in future lessons.

In [16]:
# Exporting as a CSV file
store_df.to_csv('data/optimized_gstore_data.csv', header=True, index=False)

That is it for this lesson!

As an exercise, I would like to encourage you to look at the content of each column by selecting them from the optimized DataFrame one by one. Next, create a note of how valuable the column may be in order to predict the revenue generated per customer.

This will help you to further understand the dataset and prepare yourself for the upcoming lessons.